In [10]:
#Importar la señal
%matplotlib notebook

import matplotlib.pyplot as plt
import sys
from tds_utils import *
import numpy as np
from IPython.display import Audio
import math

import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import scipy.signal as sig
import sounddevice as sd
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split





import scipy.io.wavfile as wf


class clasePalabra:

    def __init__(self):

        self.fs = 0
        self.y = 0
        self.NumVentana = 0
        self.MuestraVentana = 0
        
#Sacamos todas las palbaras por cada persona
nombrePersonas = ["Patri","Adri","Isa","Filippo","Miguelangel"]
arrayPersonas =[]
for j in range(0,len(nombrePersonas)):
    arrayPalabras = []
    i=1
    while i < 11:
        filename = nombrePersonas[j]+"/"+str(i)+".wav"
        fs,y = wf.read(filename)
        palabra  = clasePalabra()
        palabra.fs= fs
        palabra.y = y
        #Calculamos el tiempo de la grabación para poder calcular el número de ventanas y el número de muestras
        t = np.arange(0,len(palabra.y)/palabra.fs,1/palabra.fs)
        palabra.numVentana =  math.floor(max(t)/0.02)
        palabra.muestrasVentana = math.floor(len(palabra.y)/palabra.numVentana)
        arrayPalabras.append(palabra)
        i=i+1
        
    arrayPersonas.append(arrayPalabras)
    #Finalmente conseguimos un array "arrayPersonas" donde tenemos todas las palabras de cada locutor


In [14]:
arrayPalabras = []
arrayLocutores = []
#Reutilizamos nombre pero no pasa nada ya que el arrayPalabras anterior está metido en arrayPersonas

#REcorremos arrayPersonas para ir sacado las características por cada ventana de cada palabra
for x in range (0,len(arrayPersonas)):
    for z in range (0,len(arrayPersonas[x])):
        arrayLpc = []
        array = []
        i = 0
        pitch = []
        #Con este while vamos recorriendo la palabra con el número de ventanas calculado anteriormente
        while i < arrayPersonas[x][z].numVentana:
            seg = arrayPersonas[x][z].y[arrayPersonas[x][z].muestrasVentana*(i):arrayPersonas[x][z].muestrasVentana*(i+1)]
            Canal_1 = seg[:,0]
            psd1,f1 = my_spectra(Canal_1,fs)
            length = 0
            while f1[length]<0:
                length = length + 1
            psd=psd1[length:]
            f=f1[length:]
            umbral = 50
            j=3
            l=True
            lpc_f=False
            p0=13
            #Filtramos el ruido de fondo o silecio de las palabras poniendo un umbral
            #Vemos si la señal comienza a decrecer para descartar posibles pitch erróneos
            while l==True:
                if psd[j]>umbral:
                    lpc_f=True
                if psd[j+1]<psd[j]:
                    j=j+1
                else:
                    l=False
            #Los saltos en frecuecian son de 21, por lo que para filtrar la señal en el rango de 50Hz-500Hz la j la inicializamos 
            # en 3 y terminamos en 23 "aprox 500 Hz"
            while j<23:
                if psd[j]>umbral:
                    lpc_f=True
            #Aquí se compara en saltos de 2 para evitar posibles caidas no validas para el pitch verdadero
                if psd[j+2] < psd[j] and psd[j+2] > umbral :
                    pitch.append(f[j])
                    break
                j=j+1
            #Sacamos LPC
            if lpc_f==True:
                h = sig.hamming(len(Canal_1)) #hamming window
                arrayLpc.append (predlin2(Canal_1,p0,h))
            i = i+1
        media=0
        #Las medias de pitch y LPC se calculan por palabra
        media = np.mean(pitch[0:len(pitch)])
        array.append(media)
        mediaLpc = [0,0,0,0,0,0,0,0,0,0,0,0,0]
        for z in range (0,len(arrayLpc)):
            mediaLpc=mediaLpc + arrayLpc[z]
        mediaLpc= mediaLpc/len(arrayLpc)
        for p in range (0,p0):
            array.append(mediaLpc[p])
        arrayPalabras.append(array)
        arrayLocutores.append(nombrePersonas[x])

#Al final tenemos una matriz de características "arrayPalabras" y un vector de resultados "arrayLocutores"

# Creamos el modelo de NB
model_NB = GaussianNB()

#entrenamos
for i in range(0,10):
    X_train, X_test, y_train, y_test = train_test_split(arrayPalabras,arrayLocutores, test_size=0.2)

    model_NB.fit(X_train,y_train)

    
    y_hat_NB = model_NB.predict(X_test)
    print(y_hat_NB)
    print(y_test)

    #Utilizaremos y_test para comparar la estimacion con lo que realmente sabemos que hay

    #Calculamos el accuracy
    temp = y_hat_NB == y_test
    

    acc = np.mean(y_hat_NB == y_test)
    print(acc)

#Cálculos para 

['Filippo' 'Patri' 'Adri' 'Patri' 'Filippo' 'Isa' 'Filippo' 'Patri'
 'Filippo' 'Adri']
['Filippo', 'Patri', 'Adri', 'Patri', 'Miguelangel', 'Isa', 'Miguelangel', 'Patri', 'Filippo', 'Adri']
0.8
['Adri' 'Patri' 'Adri' 'Isa' 'Patri' 'Filippo' 'Miguelangel' 'Miguelangel'
 'Adri' 'Adri']
['Adri', 'Patri', 'Adri', 'Isa', 'Patri', 'Filippo', 'Miguelangel', 'Adri', 'Filippo', 'Adri']
0.8
['Patri' 'Miguelangel' 'Patri' 'Filippo' 'Patri' 'Patri' 'Isa' 'Filippo'
 'Miguelangel' 'Adri']
['Patri', 'Miguelangel', 'Patri', 'Miguelangel', 'Patri', 'Patri', 'Isa', 'Filippo', 'Miguelangel', 'Adri']
0.9
['Filippo' 'Adri' 'Adri' 'Miguelangel' 'Miguelangel' 'Isa' 'Patri' 'Patri'
 'Adri' 'Adri']
['Filippo', 'Adri', 'Adri', 'Miguelangel', 'Miguelangel', 'Isa', 'Patri', 'Patri', 'Adri', 'Adri']
1.0
['Miguelangel' 'Patri' 'Adri' 'Adri' 'Miguelangel' 'Patri' 'Patri' 'Adri'
 'Patri' 'Adri']
['Miguelangel', 'Patri', 'Adri', 'Filippo', 'Miguelangel', 'Patri', 'Patri', 'Adri', 'Patri', 'Adri']
0.9
['Patri' 'Miguela